<a href="https://colab.research.google.com/github/Stock-XAI/LLM_server/blob/main/API_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install fastapi uvicorn nest-asyncio pyngrok
!pip install transformers accelerate torch pandas yfinance bitsandbytes peft shap matplotlib

In [ ]:
import nest_asyncio
from fastapi import FastAPI, Query
from pyngrok import ngrok
from threading import Thread
import uvicorn

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import numpy as np
import re
import shap
import matplotlib.pyplot as plt

from huggingface_hub import login
from google.colab import userdata
login(token=userdata.get('HF_TOKEN'))

### 모델 로드 및 준비

In [ ]:
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    "capston-team-5/finma-7b-4bit-quantized",
    quantization_config=bnb_cfg,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, "capston-team-5/finma-7b-lora-regression-v1")
tokenizer = AutoTokenizer.from_pretrained("capston-team-5/finma-7b-lora-regression-v1")
model.eval()

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

### 프롬프트 생성 및 추론 함수

In [ ]:
def generate_prompt(ticker, interval, context, date_str):
    return (
        f"Using the context below, estimate the rate of change in the closing price of {ticker} on {date_str}.\n"
        "Return the expected value of change as a decimal.\n\n"
        "Context: date, open, high, low, close, volume, change.\n"
        f"{context}\n\nAnswer:"
    )

def generate_response(prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

def get_prompt(ticker: str, horizon_days: int):
    assert horizon_days in [1, 7, 30], "Only 1, 7, or 30 day horizon supported"
    today = datetime.today()
    predict_date = today + timedelta(days=horizon_days)
    interval, fetch_days = ("1d", 15) if horizon_days == 1 else ("1wk", 70) if horizon_days == 7 else ("1mo", 300)
    start = (today - timedelta(days=fetch_days)).strftime("%Y-%m-%d")
    end = (today + timedelta(days=1)).strftime("%Y-%m-%d")

    data = yf.download(ticker, start=start, end=end, interval=interval)[["Open", "High", "Low", "Close", "Volume"]]
    data = data.reset_index()
    data["Date"] = data["Date"].dt.strftime("%Y-%m-%d")
    data["Change"] = data["Close"].pct_change().fillna(0)

    last_10 = data.tail(10)

    def extract(val):
        return val.iloc[0] if isinstance(val, pd.Series) else val
    context = "\n".join([
        f"{extract(row['Date'])}, {float(extract(row['Open']))}, {float(extract(row['High']))}, {float(extract(row['Low']))}, "
        f"{float(extract(row['Close']))}, {int(extract(row['Volume']))}, {float(extract(row['Change'])):.6f}"
        for _, row in last_10.iterrows()
    ])

    prompt = generate_prompt(ticker, interval, context, predict_date.strftime("%Y-%m-%d"))
    response = generate_response(prompt)

    return prompt

def extract_float(answer_text):
    # 정규식: 부호/소수점 포함 실수 (가장 첫 번째만 반환)
    match = re.search(r"[-+]?\d*\.\d+", answer_text)
    if match:
        return float(match.group())
    return 0.0

def run_prediction(ticker: str, horizon_days: int):
    prompt = get_prompt(ticker, horizon_days)
    response = generate_response(prompt)
    return extract_float(response.split("Answer:")[-1].strip()), prompt, response

In [ ]:
result, prompt, response = run_prediction("018260.KS", 1)
print(result)
print(response)

[*********************100%***********************]  1 of 1 completed


0.005208
Using the context below, estimate the rate of change in the closing price of 018260.KS on 2025-06-06.
Return the expected value of change as a decimal.

Context: date, open, high, low, close, volume, change.
2025-05-22, 133400.0, 134700.0, 130400.0, 130900.0, 102772, -0.027489
2025-05-23, 130900.0, 132800.0, 130500.0, 130500.0, 73502, -0.003056
2025-05-26, 131300.0, 134400.0, 131000.0, 134400.0, 76022, 0.029885
2025-05-27, 134000.0, 135000.0, 133100.0, 133700.0, 91728, -0.005208
2025-05-28, 134400.0, 135300.0, 132700.0, 132700.0, 115382, -0.007479
2025-05-29, 134100.0, 134100.0, 130700.0, 132900.0, 110020, 0.001507
2025-05-30, 132000.0, 133200.0, 130000.0, 130200.0, 240435, -0.020316
2025-06-02, 128900.0, 133600.0, 128900.0, 132700.0, 103095, 0.019201
2025-06-04, 133100.0, 135500.0, 132400.0, 132800.0, 150383, 0.000754
2025-06-05, 131800.0, 135400.0, 131800.0, 134600.0, 108022, 0.013554

Answer: 0.005208


### SHAP/XAI 설명 함수

In [ ]:
def explain_prediction(prompt, tokenizer, model):
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)

    inputs_embeds = model.base_model.model.model.embed_tokens(input_ids)
    inputs_embeds.requires_grad_()

    outputs = model(inputs_embeds=inputs_embeds, labels=input_ids)
    loss = outputs.loss
    loss.backward()

    grads = inputs_embeds.grad.abs().sum(dim=-1).squeeze().detach().cpu().numpy()
    grads /= grads.sum()

    tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze().tolist())
    tokens = [t.replace('▁', '▁') for t in tokens]

    return grads, tokens

grads, tokens = explain_prediction(prompt, tokenizer, model)

In [ ]:
def group_tokens_to_words(tokens, scores, group_size=7):
    print("=== [Debug] group_tokens_to_words ===")
    print(f"len(tokens) = {len(tokens)}")
    print(f"len(scores) = {len(scores)}")
    if len(tokens) != len(scores):
        print("[Warning] tokens and scores length mismatch!")

    words, word_indices = [], []
    current_word, current_indices = "", []

    for i, tok in enumerate(tokens):
        if tok in ["<0x0A>", "\\n", "\n"] or re.fullmatch(r"<0x0A>", tok):
            if current_word:
                words.append(current_word)
                word_indices.append(current_indices)
                current_word, current_indices = "", []
            continue
        elif tok.startswith('▁'):
            if current_word:
                words.append(current_word)
                word_indices.append(current_indices)
            current_word = tok[1:]
            current_indices = [i]
        else:
            current_word += tok
            current_indices.append(i)
    if current_word:
        words.append(current_word)
        word_indices.append(current_indices)

    print(f"[Debug] words (len={len(words)}): {words[:10]} ...")
    print(f"[Debug] word_indices sample: {word_indices[:5]} ...")
    if word_indices:
        max_idx = max(max(idxs) for idxs in word_indices if idxs)
        print(f"[Debug] 최대 인덱스 in word_indices: {max_idx}")

    words = [re.sub(r'<0x[A-Fa-f0-9]+>', '', w) for w in words]
    words = [re.sub(r'\\n', '', w) for w in words]
    words = [w.strip() for w in words if w.strip() != '']

    # 단어별 importance 집계 (IndexError 안전 체크)
    word_scores = []
    error_count = 0
    for indices in word_indices:
        for idx in indices:
            if idx >= len(scores):
                print(f"[IndexError] idx={idx} out of range for scores (len={len(scores)})")
                error_count += 1
        safe_indices = [i for i in indices if i < len(scores)]
        score = sum(scores[i] for i in safe_indices)
        word_scores.append(score)
    if error_count:
        print(f"[Warning] 총 {error_count}건의 out of range 인덱스가 발견되었습니다.")

    words = words[35:len(words)-1]
    word_scores = word_scores[35:len(word_scores)-1]
    word_scores = [float(s) for s in word_scores]

    print(f"[Debug] 최종 words 길이: {len(words)}")
    print(f"[Debug] 최종 word_scores 길이: {len(word_scores)}")
    print(f"[Debug] word_scores 샘플: {word_scores[:10]} ...")
    print("=== [Debug] END ===\n")
    return words, word_scores


In [ ]:
ticker = 'AAPL'
horizon_days = 7
prompt = get_prompt(ticker, horizon_days)
grads, tokens = explain_prediction(prompt, tokenizer, model)
token_list, token_score_list = group_tokens_to_words(tokens, grads)

print(f"len(tokens) = {len(token_list)}")
print(f"len(grads) = {len(token_score_list)}")
print(token_list)

[*********************100%***********************]  1 of 1 completed


=== [Debug] group_tokens_to_words ===
len(tokens) = 1165
len(scores) = 1165
[Debug] words (len=106): ['<s>', 'Using', 'the', 'context', 'below,', 'estimate', 'the', 'rate', 'of', 'change'] ...
[Debug] word_indices sample: [[0], [1], [2], [3], [4, 5]] ...
[Debug] 최대 인덱스 in word_indices: 1164
[Debug] 최종 words 길이: 70
[Debug] 최종 word_scores 길이: 70
[Debug] word_scores 샘플: [0.02630615234375, 0.0232696533203125, 0.0113525390625, 0.014984130859375, 0.01690673828125, 0.00702667236328125, 0.0076751708984375, 0.02325439453125, 0.03515625, 0.02325439453125] ...
=== [Debug] END ===

len(tokens) = 70
len(grads) = 70
['2025-03-31,', '216.72579632052526,', '225.3245211987929,', '187.09465424061224,', '188.13330078125,', '366947800,', '-0.135475', '2025-04-07,', '176.96792778953252,', '200.347272648319,', '168.98840160901602,', '197.89048767089844,', '675037600,', '0.051863', '2025-04-14,', '211.16310052244776,', '212.6611361217968,', '192.11806728809685,', '196.72203063964844,', '263763500,', '-0.0059

### FastAPI 서버/엔드포인트 구현

In [ ]:
nest_asyncio.apply()
app = FastAPI()

@app.get("/predict")
def predict(ticker: str = Query(...), horizon_days: int = Query(...)):
    result, prompt, _ = run_prediction(ticker, horizon_days)
    return {
        "ticker": ticker,
        "horizon_days": horizon_days,
        "prediction_date": (datetime.today() + timedelta(days=horizon_days)).strftime("%Y-%m-%d"),
        "prediction_result": result,
        "prompt": prompt
    }

@app.get("/explain")
def explain(ticker: str = Query(...), horizon_days: int = Query(...)):
    prompt = get_prompt(ticker, horizon_days)
    grads, tokens = explain_prediction(prompt, tokenizer, model)
    token_list, token_score_list = group_tokens_to_words(tokens, grads)

    return {
        "token_list": token_list,
        "token_score_list": token_score_list
    }


### 서버 실행 & ngrok 공개

In [ ]:
ngrok_token = userdata.get('NGROK_TOKEN')
!ngrok config add-authtoken $ngrok_token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
port = 7861
Thread(target=lambda: uvicorn.run(app, host="0.0.0.0", port=port)).start()
public_url = ngrok.connect(port)
print(f"🚀 API Ready: {public_url}/predict?ticker=005930.KS&horizon_days=1")


INFO:     Started server process [538]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 7861): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


🚀 API Ready: NgrokTunnel: "https://dbde-34-125-210-219.ngrok-free.app" -> "http://localhost:7861"/predict?ticker=005930.KS&horizon_days=1


In [ ]:
import time
_time = 0
while(True):
    time.sleep(300)
    _time += 300
    print("Running Time:", _time, "sec")
    continue